In [2]:
%matplotlib inline
import math
from scipy import optimize
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt

# Inputs

In [17]:
D = 0.1731
soil = "dense sand"
psi = math.radians(32)
c = 0
gamma_bar = 18000
W_tot = 214.78
burial_depths = np.arange(0.25, 1.25, 0.25)
f = 0.6
shear_factor = 0.3
el_lengths = [0.3, 1.5, 15]

# General

In [5]:
def soil_weight(gamma, D, H):
    return gamma * D * H

soil_weights = [soil_weight(gamma_bar, D, burial_depth) for burial_depth in burial_depths]
soil_weights

[778.95000000000005,
 1557.9000000000001,
 2336.8500000000004,
 3115.8000000000002]

# Vertical Uplift

In [60]:
def vertical_uplift_mobilisation(soil, H, D):
    if "sand" in soil:
        return min(0.02 * H, 0.1 * D)

    elif "clay" in soil:
        return min(0.2 * H, 0.2 * D)

    else:
        raise ValueError("Unknown soil type.")

delta_qus = [vertical_uplift_mobilisation(soil, burial_depth, D) for burial_depth in burial_depths]
delta_qus

[0.0050000000000000001, 0.01, 0.014999999999999999, 0.017310000000000002]

In [8]:
def N_cv(c, H, D):

    if c == 0:
        return 0
    
    return min(2 * H / D, 10)
        
N_cvs = [N_cv(c, burial_depth, D) for burial_depth in burial_depths]
N_cvs

[0, 0, 0, 0]

In [9]:
def N_q(psi):
    return math.exp(math.pi * math.tan(psi)) * math.tan((math.pi / 4) + 0.5 * psi) ** 2

N_q(psi)

23.176776207012633

In [63]:
def N_qv(psi, H, D):
    
    if psi == 0:
        return 0
    
    return min(math.degrees(psi) * H / 44 / D, N_q(psi))

N_qvs = [N_qv(psi, burial_depth, D) for burial_depth in burial_depths]
N_qvs

[1.0503650018381387, 2.1007300036762775, 3.1510950055144158, 4.201460007352555]

In [64]:
def uplift_resistance(shear_factor, H, D, gamma):
    """DNV-RP-F110 2007
    """
    if "sand" in soil:
        return (1 + shear_factor * H / D) * (gamma * H * D)

    elif "clay" in soil:
        return gamma * H * D + 2 * H * c

    else:
        raise ValueError("Unknown soil type.")
  
R_maxs = [uplift_resistance(shear_factor, burial_depth, D, gamma_bar) for burial_depth in burial_depths]
R_maxs

[1116.45, 2907.8999999999996, 5374.3499999999995, 8515.7999999999993]

## RC 1

In [18]:
vertical_uplifts = [(R_max - soil_weight) for R_max, soil_weight in zip(R_maxs, soil_weights)]
vertical_uplifts

[337.5, 1349.9999999999995, 3037.4999999999991, 5399.9999999999991]

## RC 5

In [19]:
element_length = el_lengths[0]
vertical_uplifts_imp = [element_length * (R_max - soil_weight) for R_max, soil_weight in zip(R_maxs, soil_weights)]
vertical_uplifts_imp

[101.25, 404.99999999999983, 911.24999999999966, 1619.9999999999998]

In [21]:
negative_resists = [-element_length * soil_weight for soil_weight in soil_weights]
negative_resists

[-233.685, -467.37, -701.05500000000006, -934.74000000000001]

# Axial

In [24]:
def axial_soil_force(D, c, H, f, gamma_bar, psi):
    alpha = 0.608 - 0.123 * c - 0.274 / (c ** 2 + 1) + 0.695 / (c ** 3 + 1)
    K_0 = 1 - (math.sin(psi))
    return math.pi * D * alpha * c + math.pi * D * H * gamma_bar * (
        0.5 * (1 + K_0)
    ) * math.tan(f * psi)

In [28]:
def axial_mobilisation(soil):
    delta_ts = {
        "dense sand": 0.003, "loose sand": 0.005, "stiff clay": 0.008, "soft clay": 0.01
    }
    return delta_ts.get(soil, ValueError("Unknown soil type."))

delta_t = axial_mobilisation(soil)
delta_t

0.003

## RC 7

In [25]:
element_length = el_lengths[0]
T_us = [
    element_length * axial_soil_force(D, c, burial_depth, f, gamma_bar, psi)
    for burial_depth in burial_depths
]

T_us

[187.91723697750521,
 375.83447395501042,
 563.75171093251561,
 751.66894791002085]

## RC 8

In [26]:
element_length = el_lengths[1]
T_us = [
    element_length * axial_soil_force(D, c, burial_depth, f, gamma_bar, psi)
    for burial_depth in burial_depths
]

T_us

[939.58618488752609,
 1879.1723697750522,
 2818.7585546625783,
 3758.3447395501044]

## RC 9

In [27]:
element_length = el_lengths[2]
T_us = [
    element_length * axial_soil_force(D, c, burial_depth, f, gamma_bar, psi)
    for burial_depth in burial_depths
]

T_us

[9395.8618488752618,
 18791.723697750524,
 28187.585546625782,
 37583.447395501047]

# Lateral

In [33]:
def lateral_mobilisation(H, D, bound='mid'):
    
    mob = {
        'lower': 0.1 * D,
        'mid': 0.125 * D,
        'upper': 1.15 * D,
    }
    
    return mob.get(bound, ValueError("Unknown bound."))

delta_ls = [lateral_mobilisation(burial_depth, D, bound='mid') for burial_depth in burial_depths]
delta_ls

[0.0216375, 0.0216375, 0.0216375, 0.0216375]

In [30]:
def N_ch(c, H, D):

    if c == 0:
        return 0
    x = H / D
    
    return min(6.752 + 0.065 * x - 11.063 / (x + 1) ** 2 + 7.119 / (x + 1) ** 3, 9)

N_chs = [N_ch(c, burial_depth, D) for burial_depth in burial_depths]
N_chs

[0, 0, 0, 0]

In [37]:
def N_qh(psi, H, D):
    
    if psi == 0:
        return 0

    if psi < 20:
        psi = 20
    elif psi > 45:
        psi = 45
        
    psi_range = [20, 25, 30, 35, 40, 45]
    a = [2.399, 3.332, 4.565, 6.816, 10.959, 17.658]
    b = [0.439, 0.839, 1.234, 2.019, 1.783, 3.309]
    c = [-0.03, -0.09, -0.089, -0.146, 0.045, 0.048]
    d = [
        1.059 * 10 ** -3,
        5.606 * 10 ** -3,
        4.275 * 10 ** -3,
        7.651 * 10 ** -3,
        -5.425 * 10 ** -3,
        -6.443 * 10 ** -3
    ]
    e = [
        -1.754 * 10 ** -5,
        -1.319 * 10 ** -4,
        -9.159 * 10 ** -5,
        -1.683 * 10 ** -4,
        -1.153 * 10 ** -4,
        -1.299 * 10 ** -4
    ]
    x = H / D
    def par(case):
        return interp1d(psi_range, case)(psi)

    return (
        par(a) + par(b) * x + par(c) * x ** 2 + par(d) * x ** 3 + par(e) * x ** 4
    )

N_qhs = [N_qh(psi, burial_depth, D) for burial_depth in burial_depths]
N_qhs

N_qh(psi, burial_depth, D)

[2.9735646066444463,
 3.4410505180654423,
 3.8178519512614839,
 4.1185315993155598]

## RC 2 and RC 4

In [40]:
Hs = [(burial_depth + D/2) for burial_depth in burial_depths]

def lateral_soil_force(c, H, D, psi, gamma):
    return N_ch(c, H, D) * c * D + N_qh(psi, H, D) * gamma * H * D

P_us = [lateral_soil_force(c, H, D, 32, gamma_bar) for H in Hs]
P_us

[8485.5388189006953,
 17599.278939336185,
 28420.497763982647,
 40553.370302441428]

# Vertical Bearing

In [47]:
def vertical_downward_mobilisation(soil, D):
    
    if "sand" in soil:
        return 0.1 * D

    elif "clay" in soil:
        return 0.2 * D

    else:
        raise ValueError("Unknown soil type.")

delta_qd = vertical_downward_mobilisation(soil, D)
delta_qd

0.017310000000000002

In [41]:
def cot(psi):
    return 1 / math.tan(psi)

In [42]:
def N_c(psi):
    return (N_q(psi) - 1)*cot(psi)

N_c(psi)

35.49026070689833

In [43]:
def N_gamma(psi):
    return 2*(N_q(psi) + 1)*math.tan(psi)

N_gamma(psi)

30.214652959465663

## RC 3

In [57]:
def vertical_bearing_force(psi, c, D, gamma, H):

    return (
        N_c(psi) * c * D + N_q(psi) * gamma * H * D + 0.5 * N_gamma(psi) * (gamma + (1025 * 9.81)) * D ** 2
    )

Q_ds = [vertical_bearing_force(psi, c, D, gamma_bar, burial_depth) for burial_depth in burial_depths]
Q_ds

[30753.320931974013, 48806.870758426507, 66860.420584879001, 84913.97041133148]

## RC 6

In [58]:
element_length = el_lengths[0]
Q_ds = [element_length * Q_d for Q_d in Q_ds]
Q_ds

[9225.9962795922038,
 14642.061227527951,
 20058.126175463698,
 25474.191123399443]

## RC 10

In [59]:
Q_ds = [10 * Q_d for Q_d in Q_ds]
Q_ds

[92259.962795922038,
 146420.61227527951,
 200581.26175463697,
 254741.91123399444]